In [1]:
from jp_doodle import doodle_files
from jp_doodle import dual_canvas
import jp_proxy_widget
from chart_ipynb import pie
from jp_doodle import quantity_forest
from jp_doodle.quantity_forest import directory_usage
import os
from IPython.display import display
from IPython.display import HTML


class FileSystemChart:
    
    color_counter = 333
    opacity = 0.5

    def __init__(self, path, width=800, epsilon=0.02):
        path = os.path.expanduser(path)
        path = os.path.abspath(path)
        self.path = path
        self.current_path = path
        self.previous_path = path
        self.parent_path = {path: None}
        self.color_cache = {}
        self.usage_cache = {}
        self.id_to_data = {}
        self.expanded = {}
        self.epsilon = epsilon
        self.width = width
        
        members = self.directory_members(path)
        self.members = members
        labels = [d['label'] for d in members]
        color = [d['color'] for d in members]
        value = [d['size'] for d in members]
        quantity_pie = pie.Pie()
        quantity_pie.add_dataset(labels, value, 'root', color = color)
        self.widget = quantity_pie
        self.widget.setup(width=self.width)
        display(self.widget.debugging_display())
        
    
    def path_update(self, directory_name):
        path = directory_name
        if '/' not in path:
            path = self.current_path + '/' + directory_name
        else:
            directory_name = directory_name.split('/')[-1]
        if directory_name in self.usage_cache[self.current_path]:
            self.parent_path[path] = self.current_path
        self.previous_path = self.current_path
        self.current_path = path
        self.chart_update(path)
    
    def go_root(self):
        self.chart_update(self.path)
        self.previous_path = self.current_path
        self.current_path = self.path
    
    def go_previous(self):
        temp_path = self.previous_path
        self.previous_path = self.current_path
        self.current_path = temp_path
        self.chart_update(temp_path)
    
    def go_parent(self):
        temp_path = self.parent_path[self.current_path]
        if temp_path is None:
            return
        self.previous_path = self.current_path
        self.current_path = temp_path
        self.chart_update(temp_path)
    
    def chart_update(self, path):
        members = self.directory_members(path)
        self.members = members
        labels = [d['label'] for d in members]
        color = [d['color'] for d in members]
        value = [d['size'] for d in members]
        newdataset = {'label':labels,
             'data':value,
             'backgroundColor':color}
        self.widget.js_init("""
    console.log(element.chart_info.chart.config);
    element.chart_info.chart.config.data.datasets.splice(0, 1);
    element.chart_info.chart.config.data.datasets.push(newDataset);
    element.chart_info.chart.config.data.labels = newDataset.label;
    element.chart_info.chart = new Chart(element.chart_info.context, element.chart_info.chart.config);
""", newDataset=newdataset)

    
    def directory_usage(self, directory):
        cache = self.usage_cache
        if directory in cache:
            return cache[directory]
        usage = directory_usage(directory, self.epsilon)
        cache[directory] = usage
        if not usage:
            return usage
        for u in usage.values():
            u["parent"] = directory
            self.id_to_data[u["id"]] = u
        return usage
    
    def directory_members(self, directory):
        self.expanded[directory] = True
        usage = self.directory_usage(directory)
        if not usage:
            return []
        result = []
        sorter = [(u["percent"], u["name"]) for u in usage.values()]
        for (pct, filename) in reversed(sorted(sorter)):
            u = usage[filename]
            identity = u["id"]
            expanded = self.expanded.get(identity, False)
            children = None
            if expanded:
                children = self.directory_members(identity)
            r = {
                "id": identity, 
                "label": u["name"],
                "size": u["file_size"],
                "children": children,
                "expanded": expanded,
                "color": self.get_color(identity),
            }
            result.append(r)
        return result
    
    def pick_color(self):
        self.color_counter += 1
        counter = self.color_counter
        rgb = [0, 0, 0]
        for i in range(8):
            for j in range(3):
                rgb[j] = (rgb[j] << 1) | (counter & 1)
                counter = (counter >> 1)
        # darken
        for i in range(3):
            rgb[i] = (rgb[i] * 200) // 255
        return "rgba(%s,%s,%s,%s)" % (tuple(rgb) + (self.opacity,))
    
    def get_color(self, identity):
        cache = self.color_cache
        if identity in cache:
            return cache[identity]
        result = cache[identity] = self.pick_color()
        return result

In [2]:
t = FileSystemChart("../")
# t.setup_chart()

In [5]:
labels = [t.members[i]['id'] for i in range(len(t.members))]

In [6]:
def index_callback(index):
    t.chart_update(labels[index])

t.widget.js_init("""
    var canvas = element.chart_info.canvas[0];
    var chart = element.chart_info.chart;
    canvas.onclick = function(event) {
        debugger;
        console.log("onclick called" + event);
        var data = chart.getElementAtEvent(event);
        console.log("data=" + data);
        var index = data[0]._index;
        console.log("index = " + index);
        index_callback(index);
    }
""", index_callback=index_callback)